In [1]:
import os 

In [2]:
%pwd

'd:\\FYP caption Ver 1\\FYP-image-captioning-ver1-transfromers-\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd 

'd:\\FYP caption Ver 1\\FYP-image-captioning-ver1-transfromers-'

In [52]:
from dataclasses import dataclass
from pathlib import Path
@dataclass
class Data_pretrain_config:
    model: str
    n_accumulate : int 

    
    scheduler: str
    training_batch_size: int
    valid_batch_size: int
   
    
    
    #itni dair baad weights update hongay
    

    

In [53]:
from imagecaption.constants import *
from imagecaption.utils.common import read_yaml, create_directories

In [54]:
class ConfigurationManger:
    def __init__(self,config_file_path=config_file_path,params_file_path=params_file_path):
        self.config=read_yaml(config_file_path)
        self.params=read_yaml(params_file_path)
        create_directories([self.config.artifacts_root])
        


    def get_dataset_pretrain(self)->Data_pretrain_config:
        config=self.config.pre_training

        data_pretrain_config=Data_pretrain_config(
            model=self.params.TrainingArguments.model_name,
            n_accumulate=self.params.TrainingArguments.n_accumulate,
            
          
            scheduler=self.params.TrainingArguments.scheduler,
            training_batch_size=self.params.TrainingArguments.training_batch_size,
            valid_batch_size=self.params.TrainingArguments.valid_batch_size,
         
         




        )


        return data_pretrain_config


      

In [26]:
import os 
from imagecaption.logging import logger 
import torch
from torch.utils.data import  DataLoader
import tqdm
import numpy as np
import time 
import copy
from transformers import BlipForConditionalGeneration
from collections import defaultdict

In [32]:
class Pretrain():
    def __init__(self, config:Data_pretrain_config) :
        self.config=config
        self.model = BlipForConditionalGeneration.from_pretrained(self.config.model)

#pixel_values (torch.FloatTensor of shape (batch_size, num_channels, height, width)) — Pixel values. Padding will be ignored by default should you provide it. Pixel values can be obtained using BlipImageProcessor. See BlipImageProcessor.call() for details.
#output_attentions (bool, optional) — Whether or not to return the attentions tensors of all attention layers. See attentions under returned tensors for more detail.
#output_hidden_states (bool, optional) — Whether or not to return the hidden states of all layers. See hidden_states under returned tensors for more detail.
#return_dict (bool, optional) — Whether or not to return a ModelOutput instead of a plain tuple.
#interpolate_pos_encoding (bool, optional, defaults to False) — Whether to interpolate the pre-trained position encodings.
  
    def train_one_epoch(self,model,optimizer,scheduler,dataloader,device,epoch):
        model.train()
        dataset_size=0
        running_loss=0.0
        bar=tqdm(enumerate(dataloader),total=len(dataloader))
        for step,data in bar:
            input_ids=data['input_ids'].to(device)
            pixel_values =data['pixel_values'].to(device)
            batch_size=input_ids.size(0)
            outputs= model(input_ids=input_ids,pixel_values=pixel_values,labels=input_ids)

            loss =outputs.loss
            loss=loss / self.config.n_accumulate
            loss.backward()

            if (step +1 )% self.config.n_accumulate ==0:
                optimizer.step()
                # zero the parameter gradients
                optimizer.zero_grad()

                if scheduler is not None:
                    scheduler.step()
            running_loss += (loss.item() * batch_size)
            dataset_size += batch_size

            epoch_loss = running_loss / dataset_size

            bar.set_postfix(Epoch=epoch, Train_Loss=epoch_loss,
                        LR=optimizer.param_groups[0]['lr'])
       

        return epoch_loss
    @torch.no_grad()
    def valid_one_epoch(self,model,dataloader,device,epoch, optimizer):
        model.eval()
        dataset_size=0
        running_loss=0.0

        bar= tqdm(enumerate(dataloader), total=len(dataloader))
        for step , data in bar :
            input_ids=data['input_ids'].to(device)
            pixel_values=data['pixel_values'].to(device)

            batch_size=input_ids.size(0)
            outputs=model(input_ids=input_ids,pixel_values=pixel_values,labels=input_ids)
            loss=outputs.loss
            running_loss += (loss.item() * batch_size)
            dataset_size += batch_size

            epoch_loss = running_loss / dataset_size

            bar.set_postfix(Epoch=epoch, Valid_Loss=epoch_loss,
                        LR=optimizer.param_groups[0]['lr'])
        return epoch_loss


    def run_training(self,model,optimizer,scheduler, device, num_epochs ):

        if torch.cuda.is_available():
             print("[INFO] Using GPU: {}\n".format(torch.cuda.get_device_name()))   
        start=time.time()
        best_model_wtd=copy.deepcopy(model.state_dict())
        best_epoch_loss=np.inf
        history=defaultdict(list)

        for epoch in range(1,num_epochs+1):
            train_epoch_loss=self.train_one_epoch(model,optimizer,scheduler, dataloader=train_loader,device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu"))
            val_epoch_loss=self.valid_one_epoch(model,optimizer, dataloader=valid_loader, device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu"))
            

            history['Train Loss'].append(train_epoch_loss)
            history['Valid Loss'].append(val_epoch_loss)

            if val_epoch_loss<=best_epoch_loss:
                print(f"Validation Loss Improved ({best_epoch_loss} ---> {val_epoch_loss})")
                best_epoch_loss = val_epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())
                PATH = f"BestLoss.bin"
                torch.save(model.state_dict(), PATH)
                end = time.time()
        time_elapsed = end - start
        print('Training complete in {:.0f}h {:.0f}m {:.0f}s'.format(
        time_elapsed // 3600, (time_elapsed % 3600) // 60, (time_elapsed % 3600) % 60))
        print("Best Loss: {:.4f}".format(best_epoch_loss))

        # load best model weights
        model.load_state_dict(best_model_wts)

        return model, history








In [10]:
import pickle
from io import open as io_open
from torch.optim import lr_scheduler
from transformers import  AdamW

In [35]:
class Train():
    def __init__(self,config:Data_pretrain_config):
        self.config=config
        self.model = BlipForConditionalGeneration.from_pretrained(self.config.model)
        with io_open('artifacts/transformed_data_train', 'rb') as file:
            self.train_data = pickle.load(file)
        with io_open('artifacts/transformed_data_test', 'rb') as file:
            self.test_data = pickle.load(file)
    def fetch_scheduler(self,optimizer):
        sc= self.config.scheduler

        if sc == 'CosineAnnealingLR':
            scheduler = lr_scheduler.CosineAnnealingLR(optimizer,T_max=500,
                                                   eta_min=1e-6)
        elif sc == 'CosineAnnealingWarmRestarts':
            scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer,T_0=Config['T_0'],
                                                             eta_min=1e-6)
        elif sc == None:
            return None

        return scheduler
    def training(self):
        train_loader=DataLoader(self.train_data,shuffle=True, batch_size=4)
        valid_loader = DataLoader(self.test_data, shuffle=False, batch_size=8)
        model=self.model
        model.to(torch.device("cuda:0" if torch.cuda.is_available() else "cpu"))
        optimizer = AdamW(model.parameters(), lr=1e-4 , weight_decay=1e-6)
        scheduler = self.fetch_scheduler(optimizer)
        obj=Pretrain()

        model, history = obj.run_training(model, optimizer, scheduler,
                              device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
                              num_epochs=5)

        torch.save(model.state_dict(), 'artifacts/saving_trained_model')
      






    

In [56]:
try:
    config=ConfigurationManger()
    data_pretrain_config=config.get_dataset_pretrain()
    training=Train(data_pretrain_config)
except Exception as e:
    raise e
    
    

[2024-05-29 15:22:53,143: INFO: common:yaml file : config\config.yaml laoded sucessfully]
[2024-05-29 15:22:53,146: INFO: common:yaml file : params.yaml laoded sucessfully]
[2024-05-29 15:22:53,148: INFO: common:created directory at :artifacts ]


d:\FYP caption Ver 1\FYP-image-captioning-ver1-transfromers-\FYP\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
d:\FYP caption Ver 1\FYP-image-captioning-ver1-transfromers-\FYP\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\fazal\.cache\huggingface\hub\models--Salesforce--blip-image-captioning-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Wi

KeyboardInterrupt: 